In [28]:
from historic_utils import historicDownloader, historicUtils
from datetime import date, datetime
import logging

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [30]:
keywords = {'bitcoin': ['bitcoin', 'BTC'], 'dashcoin': ['dashcoin', 'DASH', 'darkcoin'], 'dogecoin': ['dogecoin', 'DOGE'], 'ethereum': ['ethereum', 'ETH'], 'litecoin': ['litecoin', 'LTC'], 'ripple': ['ripple', 'XRP'], 'monero': ['monero', 'XMR'], 'stellar': ['stellar', 'STR']}

keywordsAll = [value for key, values in keywords.items()
                       for value in values]

today = date(datetime.now().year, datetime.now().month, datetime.now().day)

detailsList = []

for key, values in keywords.items():
    
    start = {
        'bitcoin': date(2015, 1, 1),
        'dashcoin': date(2015, 5, 1),
        'dogecoin': date(2015, 1, 1),
        'ethereum': date(2015, 12, 1),
        'litecoin': date(2015, 1, 1),
        'monero': date(2015, 1, 1),
        'ripple': date(2015, 1, 1),
        'stellar': date(2015, 1, 1)
    }[key]
    
    detailsList.append({'keyword': ' OR '.join(values), 'coinname': key, 'start': start, 'end': today})

In [35]:
hd = historicDownloader(detailsList, logger=logger)
hu = historicUtils(detailsList)

{'dashcoin': 'dashcoin OR DASH OR darkcoin', 'bitcoin': 'bitcoin OR BTC', 'ethereum': 'ethereum OR ETH', 'ripple': 'ripple OR XRP', 'litecoin': 'litecoin OR LTC', 'monero': 'monero OR XMR', 'dogecoin': 'dogecoin OR DOGE', 'stellar': 'stellar OR STR'}


In [ ]:
#Series of code to download, merge, clean, fill missing, merge and clean until got all. Then move it to interpreted and clean the raw folder